In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn

import pytorch_lightning as pl
import torchmetrics

from sklearn.decomposition import PCA

In [2]:
class mnist_pl(pl.LightningModule):
    def __init__(self, inp):
        super(mnist_pl, self).__init__()
        
        self.loss = nn.CrossEntropyLoss()
        self.lr = 0.01
        
        self.train_accm = torchmetrics.Accuracy()
        self.valid_accm = torchmetrics.Accuracy()
        self.train_acc = 0.
        self.avg_train_loss = 0.
        
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(inp, 64),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(32, 10)
        )
            
    def forward(self, X):
        X = self.model(X)
        return X
    
    def configure_optimizers(self):
        optim = torch.optim.Adam(self.parameters())
        return optim
    
    def training_step(self, batch, batch_idx):
        inputs, targets = batch
        output = self(inputs)
        loss_train = self.loss(output, targets)
        predictions = torch.argmax(output, dim=1)
        acc_train = self.train_accm(predictions, targets)
        return loss_train
    
    def validation_step(self, batch, batch_idx):
        inputs, targets = batch
        output = self(inputs)
        loss_valid = self.loss(output, targets)
        predictions = torch.argmax(output, dim=1)
        acc_valid = self.valid_accm(predictions, targets)
        return {"loss": loss_valid, "p": predictions, "y": targets}
    
    def training_epoch_end(self, outputs):
        self.train_acc = self.train_accm.compute() * 100
        self.avg_train_loss = torch.stack([x['loss'] for x in outputs]).mean()
        self.train_accm.reset()
        
    def validation_epoch_end(self, outputs):
        valid_acc = self.valid_accm.compute() * 100
        avg_valid_loss = torch.stack([x['loss'] for x in outputs]).mean()
        print(f'Epoch {self.current_epoch+1}/{self.trainer.max_epochs} : Train Accuracy: {self.train_acc:.2f}%, Valid Accuracy: {valid_acc:.2f}%, Avg. Train Loss: {self.avg_train_loss:.4f}, Avg. Valid Loss: {avg_valid_loss:.4f}')
        self.valid_accm.reset()
        if self.current_epoch == self.trainer.max_epochs - 1:
            self.validation_end(outputs)
    
    def validation_end(self, outputs):
        pb = [x['p'] for x in outputs]
        yb = [x['y'] for x in outputs]
        p = torch.cat(pb, 0).view(-1)
        y = torch.cat(yb, 0).view(-1)

## Original Dataset

In [3]:
train_dataset = torchvision.datasets.MNIST(
                    root='.',
                    train=True,
                    transform=transforms.ToTensor(),
                    download=True
                    )

val_dataset  = torchvision.datasets.MNIST(
                    root='.',
                    train=False,
                    transform=transforms.ToTensor(),
                    download=True
                    )

batch_size = 128
train_loader = torch.utils.data.DataLoader(
                        dataset=train_dataset,
                        batch_size=batch_size,
                        shuffle=True,
                        num_workers=4
                        )

val_loader = torch.utils.data.DataLoader(
                        dataset=val_dataset,
                        batch_size=batch_size,
                        shuffle=False,
                        num_workers=4
                        )
trainer = pl.Trainer(max_epochs=5, num_sanity_val_steps=0)
model1 = mnist_pl(784)
trainer.fit(model1, train_dataloader=train_loader, val_dataloaders=val_loader)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores

  | Name       | Type             | Params
------------------------------------------------
0 | loss       | CrossEntropyLoss | 0     
1 | train_accm | Accuracy         | 0     
2 | valid_accm | Accuracy         | 0     
3 | model      | Sequential       | 52.6 K
------------------------------------------------
52.6 K    Trainable params
0         Non-trainable params
52.6 K    Total params
0.211     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Epoch 1/5 : Train Accuracy: 78.23%, Valid Accuracy: 91.99%, Avg. Train Loss: 0.7104, Avg. Valid Loss: 0.2734


Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Epoch 4/5 : Train Accuracy: 92.96%, Valid Accuracy: 95.86%, Avg. Train Loss: 0.2451, Avg. Valid Loss: 0.1474


Validating: 0it [00:00, ?it/s]

Epoch 5/5 : Train Accuracy: 93.69%, Valid Accuracy: 96.11%, Avg. Train Loss: 0.2228, Avg. Valid Loss: 0.1342


1

## 90% Variance

In [4]:
train_data = train_dataset.data.numpy().reshape(-1, 784)
val_data = val_dataset.data.numpy().reshape(-1, 784)

In [5]:
train_data.shape, val_data.shape

((60000, 784), (10000, 784))

In [6]:
pca = PCA(n_components=0.90)
train_data_090 = pca.fit_transform(train_data)
val_data_090 = pca.transform(val_data)
train_data_090.shape,  val_data_090.shape

((60000, 87), (10000, 87))

In [ ]:
train_data_090 = torch.from_numpy(train_data_090)
val_data_090 = torch.from_numpy(val_data_090)
train_data_090 = train_data_090.type(torch.FloatTensor)
val_data_090 = val_data_090.type(torch.FloatTensor)


train_dataset_090 = torch.utils.data.TensorDataset(train_data_090, train_dataset.targets)
val_dataset_090 = torch.utils.data.TensorDataset(val_data_090, val_dataset.targets)

In [8]:
batch_size = 128
train_loader_090 = torch.utils.data.DataLoader(
                        dataset=train_dataset_090,
                        batch_size=batch_size,
                        shuffle=True,
                        num_workers=4
                        )

val_loader_090 = torch.utils.data.DataLoader(
                        dataset=val_dataset_090,
                        batch_size=batch_size,
                        shuffle=False,
                        num_workers=4
                        )

In [9]:
trainer = pl.Trainer(max_epochs=5, num_sanity_val_steps=0)
model2 = mnist_pl(87)
trainer.fit(model2, train_dataloader=train_loader_090, val_dataloaders=val_loader_090)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores

  | Name       | Type             | Params
------------------------------------------------
0 | loss       | CrossEntropyLoss | 0     
1 | train_accm | Accuracy         | 0     
2 | valid_accm | Accuracy         | 0     
3 | model      | Sequential       | 8.0 K 
------------------------------------------------
8.0 K     Trainable params
0         Non-trainable params
8.0 K     Total params
0.032     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Epoch 1/5 : Train Accuracy: 51.52%, Valid Accuracy: 84.28%, Avg. Train Loss: 2.3768, Avg. Valid Loss: 0.5190


Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Epoch 4/5 : Train Accuracy: 86.17%, Valid Accuracy: 93.53%, Avg. Train Loss: 0.4517, Avg. Valid Loss: 0.2151


Validating: 0it [00:00, ?it/s]

Epoch 5/5 : Train Accuracy: 88.25%, Valid Accuracy: 94.37%, Avg. Train Loss: 0.3950, Avg. Valid Loss: 0.1935


1

## Only 2 Components

In [10]:
pca = PCA(n_components=2)
train_data_2 = pca.fit_transform(train_data)
val_data_2 = pca.transform(val_data)
train_data_2.shape,  val_data_2.shape

((60000, 2), (10000, 2))

In [11]:
train_data_2 = torch.from_numpy(train_data_2)
val_data_2 = torch.from_numpy(val_data_2)
train_data_2 = train_data_2.type(torch.FloatTensor)
val_data_2= val_data_2.type(torch.FloatTensor)

train_dataset_2 = torch.utils.data.TensorDataset(train_data_2, train_dataset.targets)
val_dataset_2 = torch.utils.data.TensorDataset(val_data_2, val_dataset.targets)

In [12]:
batch_size = 128
train_loader_2 = torch.utils.data.DataLoader(
                        dataset=train_dataset_2,
                        batch_size=batch_size,
                        shuffle=True,
                        num_workers=4
                        )

val_loader_2 = torch.utils.data.DataLoader(
                        dataset=val_dataset_2,
                        batch_size=batch_size,
                        shuffle=False,
                        num_workers=4
                        )

In [13]:
trainer = pl.Trainer(max_epochs=5, num_sanity_val_steps=0)
model3 = mnist_pl(2)
trainer.fit(model3, train_dataloader=train_loader_2, val_dataloaders=val_loader_2)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores

  | Name       | Type             | Params
------------------------------------------------
0 | loss       | CrossEntropyLoss | 0     
1 | train_accm | Accuracy         | 0     
2 | valid_accm | Accuracy         | 0     
3 | model      | Sequential       | 2.6 K 
------------------------------------------------
2.6 K     Trainable params
0         Non-trainable params
2.6 K     Total params
0.010     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Epoch 1/5 : Train Accuracy: 20.24%, Valid Accuracy: 36.18%, Avg. Train Loss: 5.1499, Avg. Valid Loss: 1.6554


Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Epoch 4/5 : Train Accuracy: 30.23%, Valid Accuracy: 38.89%, Avg. Train Loss: 1.7536, Avg. Valid Loss: 1.5175


Validating: 0it [00:00, ?it/s]

Epoch 5/5 : Train Accuracy: 30.51%, Valid Accuracy: 41.55%, Avg. Train Loss: 1.7453, Avg. Valid Loss: 1.5232


1